# 02 - Harmony Integration

Batch correction using Harmony (via R/rpy2).

## Workflow
1. Load merged h5ad
2. Preprocess: normalize → HVGs → regress → scale → PCA
3. Run Harmony batch correction
4. Neighbors → Leiden clustering → UMAP
5. Evaluate and visualize
6. Marker gene detection

## Outputs
- `integrated_harmony.h5ad` - Full integrated object
- `integrated_harmony_slim.h5ad` - Minimal version
- `figures/` - UMAP plots
- `markers/` - Marker gene tables

In [ ]:
import sys
sys.path.insert(0, "..")

import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import yaml

# Local utilities
from utils.preprocessing import standard_preprocess, store_raw_counts
from utils.integration import run_harmony_rpy2, compute_neighbors_and_umap, run_leiden_clustering
from utils.evaluation import summarize_integration_metrics
from utils.visualization import plot_umap_grid, plot_batch_distribution

sc.settings.verbosity = 2
sc.settings.set_figure_params(dpi=100, facecolor="white")

## Configuration

In [ ]:
# Option 1: Load from config
# config_path = "../config/my_config.yaml"
# with open(config_path) as f:
#     config = yaml.safe_load(f)

# Option 2: Define directly
config = {
    "input": {
        "h5ad_path": "./results/merged.h5ad",
        "batch_key": "dataset",
    },
    "preprocessing": {
        "n_top_genes": 3000,
        "regress_out": None,  # e.g., ["total_counts", "pct_counts_mt"]
        "max_scale_value": 10,
        "n_pcs": 50,
    },
    "integration": {
        "key": "sample_id",  # Integration variable (can differ from batch_key)
        "harmony": {
            "theta": None,  # Diversity penalty
        },
    },
    "clustering": {
        "resolutions": [0.2, 0.5, 0.8, 1.0],
        "n_neighbors": 30,
    },
    "output": {
        "dir": "./results/harmony/",
        "save_slim": True,
        "save_markers": True,
    },
}

In [ ]:
# Extract config
input_path = Path(config["input"]["h5ad_path"])
batch_key = config["input"]["batch_key"]
integration_key = config["integration"]["key"]
output_dir = Path(config["output"]["dir"])

# Create output directories
output_dir.mkdir(parents=True, exist_ok=True)
(output_dir / "figures").mkdir(exist_ok=True)
(output_dir / "markers").mkdir(exist_ok=True)

print(f"Input: {input_path}")
print(f"Batch key: {batch_key}")
print(f"Integration key: {integration_key}")
print(f"Output: {output_dir}")

## Load data

In [ ]:
print(f"Loading {input_path}...")
adata = sc.read_h5ad(input_path)
print(f"Shape: {adata.shape}")
print(f"Batches ({integration_key}): {adata.obs[integration_key].nunique()}")

In [ ]:
# Store raw counts for later
if "counts" not in adata.layers:
    print("Storing raw counts in layers['counts']...")
    store_raw_counts(adata, layer_name="counts")

## Preprocessing

In [ ]:
# Run standard preprocessing
print("Running preprocessing...")
standard_preprocess(
    adata,
    n_hvgs=config["preprocessing"]["n_top_genes"],
    regress_vars=config["preprocessing"]["regress_out"],
    n_pcs=config["preprocessing"]["n_pcs"],
    max_scale_value=config["preprocessing"]["max_scale_value"],
)

print(f"\nHVGs: {adata.var['highly_variable'].sum()}")
print(f"PCA shape: {adata.obsm['X_pca'].shape}")

In [ ]:
# Visualize PCA variance explained
sc.pl.pca_variance_ratio(adata, n_pcs=50)

## Pre-integration visualization

Check batch effects before integration.

In [ ]:
# Quick UMAP on uncorrected PCA
print("Computing UMAP on uncorrected PCA...")
sc.pp.neighbors(adata, use_rep="X_pca", n_neighbors=30)
sc.tl.umap(adata)
adata.obsm["X_umap_uncorrected"] = adata.obsm["X_umap"].copy()

In [ ]:
# Plot uncorrected
fig = plot_umap_grid(
    adata,
    color_keys=[batch_key, integration_key],
    basis="X_umap_uncorrected",
    title_prefix="Uncorrected - ",
    save_path=output_dir / "figures" / "umap_uncorrected.png",
)
plt.show()

## Harmony Integration

In [ ]:
# Load rpy2 and R harmony
%load_ext rpy2.ipython

In [ ]:
# Run Harmony
print(f"Running Harmony on {integration_key}...")
harmonized = run_harmony_rpy2(
    adata,
    batch_key=integration_key,
    use_rep="X_pca",
    theta=config["integration"]["harmony"]["theta"],
    key_added=f"X_harmony_{integration_key}",
)

print(f"Harmony embedding shape: {harmonized.shape}")

## Post-integration: Neighbors, Clustering, UMAP

In [ ]:
# Compute neighbors on harmonized embedding
harmony_key = f"X_harmony_{integration_key}"
print(f"Computing neighbors on {harmony_key}...")

sc.pp.neighbors(
    adata,
    use_rep=harmony_key,
    n_neighbors=config["clustering"]["n_neighbors"],
    metric="cosine",
)

In [ ]:
# Run Leiden clustering at multiple resolutions
print("Running Leiden clustering...")
for res in config["clustering"]["resolutions"]:
    key = f"leiden_harmony_{res}"
    sc.tl.leiden(adata, resolution=res, key_added=key)
    n_clusters = adata.obs[key].nunique()
    print(f"  Resolution {res}: {n_clusters} clusters")

In [ ]:
# Compute UMAP
print("Computing UMAP...")
sc.tl.umap(adata)
adata.obsm["X_umap_harmony"] = adata.obsm["X_umap"].copy()

## Visualization

In [ ]:
# Plot integrated UMAP by batch
fig = plot_umap_grid(
    adata,
    color_keys=[batch_key, integration_key],
    basis="X_umap_harmony",
    title_prefix="Harmony - ",
    save_path=output_dir / "figures" / "umap_harmony_batch.png",
)
plt.show()

In [ ]:
# Plot by clustering
cluster_keys = [f"leiden_harmony_{res}" for res in config["clustering"]["resolutions"]]
fig = plot_umap_grid(
    adata,
    color_keys=cluster_keys,
    basis="X_umap_harmony",
    title_prefix="",
    save_path=output_dir / "figures" / "umap_harmony_clusters.png",
)
plt.show()

In [ ]:
# Batch distribution per cluster
default_cluster = f"leiden_harmony_{config['clustering']['resolutions'][1]}"
fig = plot_batch_distribution(
    adata,
    batch_key=integration_key,
    cluster_key=default_cluster,
    save_path=output_dir / "figures" / "batch_distribution.png",
)
plt.show()

## Integration Quality Metrics

In [ ]:
# Compute metrics
metrics = summarize_integration_metrics(
    adata,
    batch_key=integration_key,
    use_rep=harmony_key,
)

print("Integration Quality Metrics:")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.3f}")

## Marker Gene Detection

In [ ]:
if config["output"]["save_markers"]:
    print(f"Finding markers for {default_cluster}...")
    sc.tl.rank_genes_groups(
        adata,
        groupby=default_cluster,
        method="wilcoxon",
    )

In [ ]:
# Plot top markers
sc.pl.rank_genes_groups(adata, n_genes=10, sharey=False)

In [ ]:
# Save marker tables
if config["output"]["save_markers"]:
    result = adata.uns["rank_genes_groups"]
    groups = result["names"].dtype.names
    
    for group in groups:
        markers_df = pd.DataFrame({
            "gene": result["names"][group],
            "score": result["scores"][group],
            "logfoldchange": result["logfoldchanges"][group],
            "pval": result["pvals"][group],
            "pval_adj": result["pvals_adj"][group],
        })
        
        # Save full table
        markers_df.to_csv(
            output_dir / "markers" / f"cluster_{group}_markers.tsv",
            sep="\t",
            index=False,
        )
        
        # Save top 50 genes
        top_genes = markers_df.head(50)["gene"].tolist()
        with open(output_dir / "markers" / f"cluster_{group}_top50.txt", "w") as f:
            f.write("\n".join(top_genes))
    
    print(f"Saved markers for {len(groups)} clusters")

## Save Results

In [ ]:
# Save full object
full_path = output_dir / "integrated_harmony.h5ad"
print(f"Saving full object to {full_path}...")
adata.write_h5ad(full_path)
print("Done!")

In [ ]:
# Save slim version (counts, obs, obsm only)
if config["output"]["save_slim"]:
    slim_path = output_dir / "integrated_harmony_slim.h5ad"
    print(f"Saving slim object to {slim_path}...")
    
    adata_slim = sc.AnnData(
        X=adata.layers["counts"] if "counts" in adata.layers else adata.X,
        obs=adata.obs,
        var=adata.var[["highly_variable"]],
        obsm=adata.obsm,
    )
    adata_slim.write_h5ad(slim_path)
    print("Done!")

In [ ]:
# Save cell metadata separately
metadata_path = output_dir / "cell_metadata.tsv"
adata.obs.to_csv(metadata_path, sep="\t")
print(f"Saved metadata to {metadata_path}")

## Summary

Integration complete. Outputs saved to the output directory.

### Next Steps
- Review UMAP plots and batch mixing
- Annotate clusters based on marker genes
- Compare with other integration methods if needed